<a href="https://colab.research.google.com/github/Aaaaangla/WeatherWise_Angla_Li/blob/main/WeatherWise_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook
Welcome to **WeatherWise**! This is a weather advisor that provides comprehensive visualisations and supported chat box conversation with your customised AI assistant.

还可以添加更多的descriptions。

## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

可以添加更多的descriptions。

In [ ]:
!pip install pyinputplus
!pip install gradio

  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/f9/a5/08a02a12331517acd02bfd9e44a940dd141197e2dc527f7a4cfc791268de/gradio-5.46.1-py3-none-any.whl.metadata
  Using cached gradio-5.46.1-py3-none-any.whl.metadata (16 kB)
  Obtaining dependency information for fastapi<1.0,>=0.115.2 from https://files.pythonhosted.org/packages/6d/45/d9d3e8eeefbe93be1c50060a9d9a9f366dba66f288bb518a9566a23a8631/fastapi-0.117.1-py3-none-any.whl.metadata
  Using cached fastapi-0.117.1-py3-none-any.whl.metadata (28 kB)
  Obtaining dependency information for ffmpy from https://files.pythonhosted.org/packages/74/d4/1806897b31c480efc4e97c22506ac46c716084f573aef780bb7fb7a16e8a/ffmpy-0.6.1-py3-none-any.whl.metadata
  Using cached ffmpy-0.6.1-py3-none-any.whl.metadata (2.9 kB)
  Obtaining dependency information for gradio-client==1.13.1 from https://files.pythonhosted.org/packages/9a/02/dd4ecbcf654abc4b969f84def5735b7570224e9590912523b9f3eaf0d741/gradio_client-1.13.1-py3-none

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2025.9.0 which is incompatible.


In [ ]:
import os

# Prompt user to enter their API key
os.environ["OPENWEATHER_API_KEY"] = input("Enter your OpenWeatherMap API key: ")
API_KEY = os.environ["OPENWEATHER_API_KEY"]

Enter your OpenWeatherMap API key: fb1582724c9b13d13d1b7420b360c14e


## 📦 Setup and Configuration
Import required packages and setup environment.
稍微改一下description。

In [ ]:
import requests
import pandas as pd

# Graphs
import matplotlib.pyplot as plt
import seaborn as sns

# Chatbot
import re
import nltk

# UI design
from ipywidgets import VBox, HBox, Dropdown, Button, Output
import gradio as gr
import pyinputplus as pyip

In [ ]:
# Reduce repeat updates if the API doman ever changes
API_BASE = 'https://api.openweathermap.org'

# Ensure consistent units
UNITS = 'metric' # Celsius (°C), meters/second (m/s)

# Prevent reuqest long request time
DEFAULT_TIMEOUT = 10 # seconds

In [ ]:
# Brief test for nltk
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\angla\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [ ]:
# Smoke test for vertify the correction of API key, network connection, and API server
def _smoke_test(city="Sydney"):
    key = os.environ.get("OPENWEATHER_API_KEY")
    assert key, "OPENWEATHER_API_KEY not set"
    r = requests.get(
        f"{API_BASE}/data/2.5/weather",
        params={"q": city, "appid": key, "units": UNITS},
        timeout=DEFAULT_TIMEOUT
    )
    r.raise_for_status()
    data = r.json()
    print("OK:", data["name"], "-", data["weather"][0]["description"])

_smoke_test('Perth')

OK: Perth - clear sky


## 🌤️ Weather Data Functions
写一点description在这里。

##### Logic for get_weather_data(location, forecast_days=5)
1. Read API Key and constants.
    - the API key, API_BASE, UNITS, DEFAULT_TIMEOUT
2. Validate input
    - the forecast day is between 1 - 5; if false, show ValueError
3. Request current weather
    - /data/2.5/weather is the endpoint, with parameters, may need extract temperature, description, humidity, wind speed, etc.
4. Request forecast data
    - /data/2.5/forecast is the endpoint (3-hourly forecast) and keep only forecast_days worth of data
5. Format results into dictionary
6. Handle errors gracefull
    - Use try/except for network errors (requests.exceptions), and raise clear messages if API key is missing or location is invalid.

In [ ]:
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    pass

In [ ]:
import os
import requests
from collections import Counter, defaultdict
from datetime import datetime
import pandas as pd  # optional, only needed for the to_dataframe helper

API_BASE = 'https://api.openweathermap.org'
UNITS = 'metric'          # °C, m/s
DEFAULT_TIMEOUT = 10      # seconds


class WeatherError(RuntimeError):
    """Raised for predictable, user-facing weather errors."""


def _get_api_key() -> str:
    key = os.environ.get("OPENWEATHER_API_KEY")
    if not key:
        raise WeatherError("OPENWEATHER_API_KEY not set. Please set it in the environment.")
    return key


def _fetch_json(path: str, params: dict) -> dict:
    """Minimal wrapper around requests with good messages."""
    url = f"{API_BASE}{path}"
    try:
        r = requests.get(url, params=params, timeout=DEFAULT_TIMEOUT)
        r.raise_for_status()
        data = r.json()
        # OpenWeather sometimes returns 200 with a message; guard for that:
        if isinstance(data, dict) and data.get("cod") not in (200, "200"):
            # e.g., {"cod":"404","message":"city not found"}
            raise WeatherError(f"API error ({data.get('cod')}): {data.get('message')}")
        return data
    except requests.exceptions.RequestException as e:
        raise WeatherError(f"Network/API request failed: {e}") from e


def _summarize_forecast_by_day(items, tz_offset_seconds: int):
    """
    items: list of 3-hour forecast entries as returned by /forecast
    tz_offset_seconds: timezone shift for the location (from 'city.timezone')
    """
    # bucket entries by local calendar day
    buckets = defaultdict(list)
    for row in items:
        # dt is UTC seconds; shift to local then take date
        local_dt = datetime.utcfromtimestamp(row["dt"] + tz_offset_seconds)
        day_key = local_dt.date().isoformat()
        buckets[day_key].append(row)

    summaries = []
    for day, rows in sorted(buckets.items()):
        temps = [r["main"]["temp"] for r in rows]
        tmins = [r["main"]["temp_min"] for r in rows]
        tmaxs = [r["main"]["temp_max"] for r in rows]
        hums = [r["main"]["humidity"] for r in rows]
        winds = [r["wind"]["speed"] for r in rows]

        # most frequent weather description of the day
        descs = [r["weather"][0]["description"] for r in rows if r.get("weather")]
        common_desc = Counter(descs).most_common(1)[0][0] if descs else ""

        # precipitation volume in mm (rain/snow fields are optional)
        def get_mm(r, field):
            return float(r.get(field, {}).get("3h", 0.0))
        rain_mm = sum(get_mm(r, "rain") for r in rows)
        snow_mm = sum(get_mm(r, "snow") for r in rows)

        summaries.append({
            "date": day,
            "temp_min": round(min(tmins), 1),
            "temp_max": round(max(tmaxs), 1),
            "temp_avg": round(sum(temps)/len(temps), 1),
            "humidity_avg": round(sum(hums)/len(hums), 0),
            "wind_avg": round(sum(winds)/len(winds), 1),
            "description": common_desc,
            "rain_mm": round(rain_mm, 1),
            "snow_mm": round(snow_mm, 1),
        })
    return summaries


def get_weather_data(location: str, forecast_days: int = 5) -> dict:
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name (e.g., "Perth", "Sydney,AU").
        forecast_days (int): Number of days of forecast to return (1–5 for free tier).

    Returns:
        dict: {
          "location": str,
          "coords": {"lat": float, "lon": float},
          "current": {...},
          "forecast": [ {date, temp_min, temp_max, ...}, ... up to forecast_days ]
        }
    """
    if not (1 <= int(forecast_days) <= 5):
        raise WeatherError("forecast_days must be between 1 and 5 for the free 5-day/3-hour API.")

    key = _get_api_key()

    # 1) current conditions
    current = _fetch_json(
        "/data/2.5/weather",
        {"q": location, "appid": key, "units": UNITS}
    )
    city_name = current.get("name", location)
    coord = current.get("coord", {})
    tz_offset = current.get("timezone", 0)

    current_block = {
        "temperature": current["main"]["temp"],
        "feels_like": current["main"]["feels_like"],
        "temp_min": current["main"]["temp_min"],
        "temp_max": current["main"]["temp_max"],
        "pressure": current["main"]["pressure"],
        "humidity": current["main"]["humidity"],
        "wind_speed": current["wind"]["speed"],
        "clouds_pct": current.get("clouds", {}).get("all", 0),
        "description": current["weather"][0]["description"] if current.get("weather") else "",
        "observed_at_utc": datetime.utcfromtimestamp(current["dt"]).isoformat() + "Z",
    }

    # 2) forecast (3-hr steps → summarize by day)
    forecast_raw = _fetch_json(
        "/data/2.5/forecast",
        {"q": location, "appid": key, "units": UNITS}
    )
    city_block = forecast_raw.get("city", {})
    tz_offset = city_block.get("timezone", tz_offset)  # prefer forecast's city timezone
    daily = _summarize_forecast_by_day(forecast_raw.get("list", []), tz_offset_seconds=tz_offset)

    result = {
        "location": city_name,
        "coords": {"lat": coord.get("lat", city_block.get("coord", {}).get("lat")),
                   "lon": coord.get("lon", city_block.get("coord", {}).get("lon"))},
        "timezone_shift_seconds": tz_offset,
        "current": current_block,
        "forecast": daily[:forecast_days],
    }
    return result


# ------- Optional helpers (handy in notebooks/UIs) -------

def weather_to_dataframe(weather_dict: dict) -> pd.DataFrame:
    """Convert the forecast list into a DataFrame for quick plotting/display."""
    return pd.DataFrame(weather_dict["forecast"])


def pretty_print_weather(weather_dict: dict):
    """Tiny text view for debugging."""
    loc = weather_dict["location"]
    cur = weather_dict["current"]
    print(f"{loc}: {cur['description']} | {cur['temperature']}°C "
          f"(feels {cur['feels_like']}°C), wind {cur['wind_speed']} m/s")
    for d in weather_dict["forecast"]:
        print(f"  {d['date']}: {d['description']} "
              f"{d['temp_min']}–{d['temp_max']}°C, "
              f"rain {d['rain_mm']} mm, wind ~{d['wind_avg']} m/s")


In [ ]:
data = get_weather_data("Perth", forecast_days=5)
pretty_print_weather(data)

df = weather_to_dataframe(data)
df  # or plot df["date"] vs df["temp_max"] with matplotlib

Perth: clear sky | 22.29°C (feels 22.06°C), wind 3.35 m/s
  2025-09-21: clear sky 17.1–22.5°C, rain 0.0 mm, wind ~3.4 m/s
  2025-09-22: clear sky 13.6–24.4°C, rain 0.0 mm, wind ~4.4 m/s
  2025-09-23: clear sky 15.8–25.5°C, rain 0.0 mm, wind ~4.1 m/s
  2025-09-24: scattered clouds 16.9–25.9°C, rain 0.0 mm, wind ~2.9 m/s
  2025-09-25: overcast clouds 17.0–22.3°C, rain 0.0 mm, wind ~3.6 m/s


,date,temp_min,temp_max,temp_avg,humidity_avg,wind_avg,description,rain_mm,snow_mm
0,2025-09-21,17.1,22.5,20.9,59.0,3.4,clear sky,0.0,0.0
1,2025-09-22,13.6,24.4,19.0,50.0,4.4,clear sky,0.0,0.0
2,2025-09-23,15.8,25.5,20.5,46.0,4.1,clear sky,0.0,0.0
3,2025-09-24,16.9,25.9,21.4,44.0,2.9,scattered clouds,0.0,0.0
4,2025-09-25,17.0,22.3,19.5,59.0,3.6,overcast clouds,0.0,0.0
